# Optimal Bayesian Experimental Design

R. D. McMichael 
rmcmichael@nist.gov  
National Institute of Standards and Technology  
Gaithersburg, MD  USA
March 29, 2019

## Contents

 - [Introduction](#introduction)
 - [Philosophy and goals](#philosophy)
 - [Requirements](#requirements)
 - [Demos](#demos)
 - [Theory of Operation](#theory)
 - [Software documentation](#software)

<a id="introduction"></a>
## Introduction

This manual describes an implementation of optimal Bayesian experimental design methods.  These methods address routine measurements where data are fit to experimental models in order to obtain model parameters.  The twin benefits of these methods  are reduced uncertainty with fewer required measurements.  These methods are therefore most beneficial in measurements where measurements are expensive in terms of money, time, risk, labor and/or discomfort.  The price for these benefits lies in the complexity of automating such measurements and in the computational load required.  It is the goal of this package to assist potential users in overcoming at least the programming hurdles.

Optimal Bayesian experimental design is not new, at least not in the statistics community.  A review paper from 1995 by [Kathryn Chaloner and Isabella Verinelli](https://projecteuclid.org/euclid.ss/1177009939) reveals that the basic methods had been worked out in preceding decades.  The methods implemented here closely follow [Xun Huan and Youssef M. Marzouk](http://dx.doi.org/10.1016/j.jcp.2012.08.013) which emphasizes simulation-based experimental design.  Optimal Bayesian experimental design is also an active area of research

There are at least three important factors that encourage application of these methods today.  First, the availability of flexible, modular (package friendly?) computer languages such as Python.  Second, availability of cheap computational power.   Most of all though, an increased awareness of the benefits of code sharing and reuse is growing in scientific communities, and the sharing is facilitated by websites such as sourceforge, bitbucket and github.


<a id="philosophy"></a> 
## Philosophy and goals

> If it sounds good, it is good
>> Duke Ellington

Of course, jazz legend Duke Ellington was talking about music, where it's all about the sound.  For this package, it's all about being useful.  The goals are modest: to adapt some of the developments in optimal Bayeseian experimental design research for practical use in laboratory settings, giving users tools to make better measurements. 

- If it's a struggle to use, it can't run good.
- If technical jargon is a barrier, it can't run good
- If the user finds it useful, it runs good.
- If it runs good, it is good.

<a id="requirements"></a>
## Requirements for users

It takes a little effort to get this software up and running.  Here's what a user will need to supply to get started.

1. Python 3.x
2. numpy python package
2. An experiment that yields measurement results with uncertainty estimates. 
3. A parametric model for the experiment - typically a function with parameters to be determined. 
4. A working knowledge of Python programming - enough to follow examples and program your own model.


<a id="demos"></a>
## Demos

### Locating a Lorentzian peak

![Comparison of measure-then-fit and OBED method](images/demoLorentzfig1.png)

Figure 1:  A comparison of measure-then-fit (left) and optimal Bayesian experimental design (right).  Both methods measure the same "true" peak with Gaussian noise added ($\sigma = 1$) independently.  The peak parameters are selected randomly: center between 2 and 4, height between 2 and 5, background between -1 and 1 and peak width between 0.1 and 0.3.  On the left, 30 evenly-spaced "measurements" are made and fit using `scipy.optimize.curve_fit()`.  A curve using the best-fit parameters is plotted for comparison with the true curve.  The diagonal element of the covariance matrix is taken as the square of the uncertainty.   On the right, the optimal Bayes experimental design method is used sequentially.  Iterations stop when the standard deviation of the `x0` peak center distribution is less than the uncertainty of the fit on the left.  Green curves correspond to random draws from the parameter distribution at the stopping point. Typical runs of this example require something like 1/4 to 1/2 as many measurements.  See `Demos/demoLorentzian.py`.

### 10 $\times$ measurement speedup

![Mechanism of speedup](images/rootN.png)

Figure 2: A different view of the Lorentzian peak problem, contrasting efficiency differences between methods.   The left panel shows results after 1000 measurements of a spectrum.  The simulated experimental noise is Gaussian with standard deviation $\sigma = 1$. In the "average & fit" method, 50 simulated measurements are averaged at each of 20 points, yielding a uniform uncertainty of $\sigma_y = 1/\sqrt{50}$.  In the OptBayesExpt method, the algorithm focuses attention on the sides of the peak, as shown in the histogram.  The symbol areas are proportional to the weights of the data points, $1/\sigma^2$.  The smallest points correspond to $\sigma_y = 1.0$, and the largest to $\sigma_y \approx 0.055.$  The right panel plots the evolution of uncertainty in the peak center $x_0$ with the number of accumulated measurements.  Both the average & fit technique and the OptBayesExpt method scale like $\sigma_{x0} \propto 1/\sqrt{N}$ ($N$ = measurement number) for large $N$, but the OptBayesExpt requires approximately 10 times fewer measurements to achieve the same uncertainty.  

Details: For the OptBayesExpt results, the uncertainty is the standard deviation of the parameter distribution function, with peak height and background treated as nuisance variables.  The average & fit method used `scipy.optimize.curve_fit()`.  The uncertainty plotted here is the square root of the diagonal element in the covariance matrix.  The peak height, background level and peak center are treated as unknowns, and the half-width line width is fixed at 0.3. Repeated runs show a variety of OptBayesExpt behaviors with the initial measurements, which is typically followed by a rapid decrease in the uncertainty and a transition to $1/\sqrt{N}$ behavior. See `Demos/demoLorentzian2.py`.

### Tuning a $\pi$ pulse

![measurements of transition probability vs pulse length and detuning](images/pipulsefig2.png)

Figure 3: A $\pi$ pulse is a method of inverting spins that is frequently used in nuclear magnetic resonance (NMR and MRI) and pulsed electron paramagnetic resonance (EPR).  In order to be accurate, the duration and frequency of the pulse must be tuned.  On the left, the background image displays the model photon counts for optically detected spin manipulation for different detunings and pulse lengths.  White indicates the expected result for spin up and black, spin down.  Points indicate simulated measurement settings, with sequence in order from white to dark red.  Simulated measurements have 1$\sigma$ uncertainties of 100.  The right panel shows the probability distribution function after 50 measurements, with the red dot at the simulated "true" value.  The gray line shows the path of the probability maximum.  See `Demos.pipulse.py`.

### Slope Intercept
![Straight line measurement examples](images/slopeintercept.png)

Figure 4: This demo uses a straight line model, a case where we know ahead of time where the "best" measurements should be made.  Measurements at the ends of a line are the most effective at reducing uncertainty in the slope and intercept values.  But perhaps we also need to be convinced or reassured that the straight line model is appropriate, and we'd like to have some data in the middle, too. The OptBayesExpt class provides two methods for flexibility in measurement selection.  The `opt_setting()` method selects the setting with the highest _utility_ $\max[U(x)]$.  The first panel shows that it behaves as expected, choosing measurements at the ends of the line.  The `good_setting()` method is more flexible, selecting settings with a probability based on the _utility_ and the `pickiness` parameter.  The 2nd through 4th panels show that the `good_setting()` algorithm selects more diverse setting values as the `pickiness` is reduced.  Note also that the standard deviations increase from left to right as measurement resources are diverted away from reducing uncertainty.  Each run uses 40 points.  See `Demos/slopeintercept.py`.

<a id="theory"></a>
## Theory of operation

The optimal Bayes experimental design method incorporates two main jobs, which we can describe as "learning fast" and "making good decisions"

### Learning fast

The learning process is a straightforward application of the well-known Bayesian inference method.  If that last sentence made perfect sense to you, feel free to skip ahead.  

We do measurements in order to learn things.  At the very beginning, before we start measuring, we may have some knowledge or experience, but we expect to have better information after we measure. Let's start in the logical place, ~~at the beginning~~ in the middle, asking how the knowledge is refined as a new measurement result is digested.  For that we need to use some technical language.

We'll express our knowledge of the set of model parameters $\theta$ as a probability distribution function $p(\theta)$.  If $p(\theta)$ is a broad distribution, then we really don't know the values very well, and if $p(\theta)$ is narrow, the uncertainty is small.  New measurement results $m$ should allow us to refine $p(\theta)$; we want to know the new probability distribution $p(\theta|m)$ after we have taken $m$ into account.  The vertical bar in the notation $p(\theta|m)$ indicates a conditional probability, the distribution of $\theta$ values given $m$. 

Bayes rule gives us
  $$ p(\theta|m) = \frac{p(m|\theta) p(\theta)}{p(m)}. $$

It's easy to write down Bayes rule. You just put the symbols in the right places.  On the other hand, understanding the symbols can be tricky, so let's pick it apart.   All of the terms here have technical names.  The left side is the _posterior_ distribution, i.e. the distribution of parameters $\theta$ after we include $m$. On the right, distribution $p(\theta)$ is the _prior_, representing what we knew about the parameters $\theta$ before the measurement. In the denominator, $p(m)$ is the _evidence_, which has no $\theta$ dependence, so it behaves like a normalization constant.  In this case we can ignore the _evidence_ if we promise to normalize our _posterior_ before we use it.  (Edit this)

The term that we will focus on, $p(m|\theta)$, is called the _likelihood_. It's the probability of getting measurement $m$ given variable parameter values $\theta$. Just as any conditional probability $p(a|b)$ depends on both $a$ and $b$, $p(m |\theta)$ depends on both $m$ and $\theta$.  But when we put it into practice, we're going to have fixed measurement results $m_i$ to "plug in" for $m$.  It's important to keep sight of the fact that $p(m_i|\theta)$ is still a function of theta. Conceptually, we can try out different parameter values in our model to produce a variety of measurement predictions.  Some parameters will produce predictions closer to $m_i$ (and more likely) and for other parameters, predictions will be further away (and less likely).

To go further, we need to specify what we mean by a measurement $m_i$.  Of course the measurement includes the "value" $y_i$ (which could be more than one number), but we also require it to include measurement settings $x_i$ and an estimate of the uncertainties $\sigma_i$. The choice to require uncertainties adds responsibility to the experiment, while making the model simpler. A supremely complete model would include not only the theory of our system's behavior, but also a model of the measurement noise, environmental fluctuations, the universe and everything.  It's a bit much to ask.  By requiring uncertainty estimates in the measurement, the experimental model can be an easy-to-program deterministic function $y(x,\theta)$ rather than a distribution.  The probabilistic nature of measurement noise (and everything) is captured by the $\sigma_i$, which provides a recipe for a Gaussian distribution.

Unpacking a mean value $y_i$ and uncertainty $\sigma_i$ from $m$, we can supplement the simple model with the measured uncertainty to yield the _likelihood_

  $$ p(m_i|\theta) = \frac{1}{\sqrt{2\pi}\sigma_i} \exp\left[-\frac{[y_i - y(x_i, \theta)]^2 }{ 2\sigma_i^2 } \right]$$.
  
Now we know how to update our "knowledge" of parameters $\theta$ expressed as a probability distribution $P(\theta)$.
1. Collect measurement data including settings, $x_i$, measurement values $y_i$ and measurement uncertainties $\sigma_i$.
2. For all parameter combinations $\theta$ calculate the model's prediction of the mean measurement $y(x_i, \theta)$
3. For all parameter combinations $\theta$ multiply $p(\theta)$ by the likelihood $\exp[-(y_i-y(x, \theta))^2/2\sigma_i^2 ]$ 

As we make more measurements, we'll update $p(\theta|m_i)$ to $p(\theta|m_i, m_j, \ldots)$ and so on.  In order to keep the notation readable, we'll adopt a convention that $p(\theta)$ always represents the most up-to-date parameter distribution that we have.

We just made several important assumptions:
 - That our model function $y(x, \theta)$ is a good description of our system, and
 - that the noise in our measurement is Gaussian with standard deviation $\sigma_i$.

On one hand we have to admit that these assumptions don't allow us to address all important cases.  On the other hand, these are the same assumptions we often make in doing least-squares curve fitting.


### Making good decisions

The next important job in the process is figuring out the settings for the next measurement that will best advance our goals.   At least part of our goal is to make the parameter probability distribution $p(\theta)$ narrow while minimizing cost or time spent.  But we might have more than one goal: of course to find the best parameter values, but maybe also to convince a skeptical journal reviewer or to get project funding.

The challenge, then is to develop a _utility function_ $U(x)$ that helps us to predict and compare the relative benefits of different possible experimental settings $x$.

First, an appeal to intuition.  Our system model describes a connection between parameter values $\theta$ and measurement results $y$, so if we try out different parameter values the model will predict different measurement outcomes.  Intuitively, if we want to constrain the parameter values, it would do the most good to "pin down" the measurement by selecting the settings $x$ where the predicted $y$ has the largest variations.  To be smart about the selection, we need to take advantage of the accumulated measurements, so we only try out parameters that are samples from the $p(\theta)$ parameter distribution.

In broad strokes, our approach to making good decisions about measurement settings goes like this:
1. For random draws $\theta_i$ of parameters from the distribution $p(\theta)$
   Use the model to predict $y_i = y(x,\theta_i)$ for every possible setting $x$.
2. Calculate a measure of the spread in $y_i$ values
3. Pick a measurement setting with a large spread.

#### Estimate benefits

To translate such a qualitative argument into code, a good place to start is to clarify what we mean by "doing the most good" in refining our parameter distribution $p(\theta)$.  When we determine model parameters, usually the goal is to get results with small uncertainty.  But here we're thinking in terms of a possibly multidimensional distribution $p(\theta)$.  Information theory gives us information entropy as a way to quantify the sharpness of a probability distribution.  The information entropy of a probability distribution $p(a)$ is defined as  
 $$ E = -\int da\; p(a)\; \ln[p(a)] $$  
Note that the integrand above is zero for both $p(a) = 1$ and $p(a)=0$.  It's the intermediate values encountered in a spread-out distribution where the information entropy accumulates.  For common distributions, like rectangular or Gaussian, that have characteristic widths $w$ the entropy goes like $\ln(w) + C$.

We adopt the information entropy as our measure of $p(\theta)$ sharpness, and that makes it possible to estimate how much $E$(posterior) - $E$(prior) we might get for predicted measurement values $y$ at different settings $x$.  Actually, the statisticians use something slightly different called the Kulback-Liebler divergence:
$$ D^{KL}(y,x) = \int d\theta\; p(\theta |y,x)\ln \left[ \frac{p(\theta | y,x)}{p(\theta)}\right]. $$  
In this expression $p(\theta | y,x)$ is a speculative parameter distribution we would get if we happened to measure a value $y$ using settings $x$.  By itself, $D^{KL}(y,x)$ doesn't work as a utility function $U(x)$ because it depends on this arbitrary possible measurement value $y$.  So we need to average $D^{KL}$, weighted by the probability of measuring $y$.
$$ U(x) = \int dy \int d\theta\; p(y|x) p(\theta |y,x)\ln \left[ \frac{p(\theta | y,x)}{p(\theta)}\right]. $$  

The result for each potential setting $x$ is the difference in information entropy-like terms for two distributions:
1. The information entropy of $p(y|x)$, the distribution of measurement values expected at setting $x$.  Strangely, an important property of $p(y|x)$ doesn't appear in the notation: that it includes likely variations of $\theta.$  Explicitly,
  $$ p(y|x) = \int d\theta'\; p(\theta') p(y|\theta',x) $$
2. The other distribution, $p(y|\theta,x)$ is the distribution when $\theta$ and $x$ are fixed.  The entropy of this distribution is averaged over $\theta$ values. 
  $$ \int d\theta\; p(\theta) \int dy\; p(y|\theta,x) \ln [ p(y|\theta, x) ] $$

Term 1 is the entropy of the $\theta$-averaged $y$ distribution and Term 2 is the $theta$ average of the entropy of the $y$ distribution.  Loosely, Term 1 is a measure of the spread in $y$ values due to both measurement noise and likely parameter variations, while  term 2 is (mostly) just the measurement noise.

If we were to implement this result directly, the next step would be to estimate the entropy integrals using random draws from $p(\theta)$. To do this, we would need to make model measurement results for enough samples from $p(\theta)$ to make a good estimate of $p(y|\theta,x)$, and that task is computationally intensive.  It takes a lot of samples to get smooth distributions.

In keeping with our "runs good" philosophy, let's consider _approximate methods_ (which is fancy talk for "cheating.")  What are the risks?  All we require of our decision-making algorithm is that is gives us smart, data-driven decisions. Is it critical that we make the absolute best measurement every single time? Probably not.  We don't need precise values, we just need to compare the utility of different settings.  Even if we don't choose the absolute best setting, a "pretty good" choice will do more good than an uninformed choice.  The only really bad possibility is the risk that the software will run too slowly to be useful.   

Having given ourselves permission to be a little sloppy, let's approximate and see what happens if we assume all of the distributions are normal (Gaussian).  The information entropy of the normal distribution has a term that goes like $\ln$(width).  Term 1 from above is a convolution of the measurement noise distribution (width = $\sigma_y$ and the distribution of model $y$ values (width = $\sigma_{y,\theta}$) that reflects the connection to the parameter distribution.  A property of normal distributions is that a convolution of normal distributions is another normal distribution with width = $\sqrt{\sigma_{y,\theta}^2 + \sigma_y^2}$.  Under the assumption of normal distributions, we now have an approximate utility function

 $$ U^*(x) \approx \ln(\sqrt{\sigma_\theta^2 + \sigma_y^2}) - \ln(\sigma_y) 
         = \frac{1}{2}\ln\left[\frac{\sigma_{y,\theta}(x)^2}{\sigma_y(x)^2}+1\right]$$

This approximation has some reasonable properties. The dependence on $\sigma_{y,\theta}$ matches our initial intuition that high-utility parameters are those where measurements vary a lot due to parameter variations.  The dependence on measurement noise $\sigma_y$ also has an intuitive interpretation: that it's less useful to make measurements at settings $x$ where the instrumental noise is larger.  This approximate utility function is also positive, i.e. more data helps narrow a distribution.   
Finally, $U^*(x)$, which is an approximate information entropy change, has the property that the parameter distribution width $\sigma_\theta$ behaves asymptotically like $N^{-1/2}$ after $N$ iterations when noise dominates.  At least in one dimension, 
  $$ d \ln(\sigma_\theta) = \frac{1}{2}\frac{\sigma_{y,\theta}(x)^2}{\sigma_y(x)^2}dN. $$
The left side is the approximate entropy change in one iteration ($dN = 1$).  If the parameter variations are small enough, $\sigma_{y,\theta} \approx dy/d\theta\; \sigma_\theta$.  Then the differential equation implied above has the solution
  $$ \sigma_\theta \propto N^{-1/2} $$
which is typical "beating down the noise" behavior of long-term averaging.


#### Estimate the costs

There are two very important questions that we have left unresolved:
1. What if some settings are more difficult/time consuming/expensive than others?
2. When should I quit measuring?

Both of these questions involve weighing the cost of a measurement against the expected benefits, and that's something that we just can't do, because a) we don't know how and b) it'd be a legal mess if we claimed that we did.   Economic decisions are left to the user.


<a id="software"></a>
## Software documentation

- **OptBayesExpt.py** [[notebook]](OptBayesExpt.ipynb)[[html]](OptBayesExpt.html)
- **ProbDistFunc_class.py** [[notebook]](ProbDistFunc.ipynb)[[html]](ProbDistFunc.html)
- **ExptModel_class.py** [[notebook]](ExptModel_class.ipynb)[[html]](ExptModel_class.html)

## Missing pieces

A. What if the output of the model is a probability distribution instead of a number, and the likelihood distribution $p(m|\theta)$ is spread out by more than measurement uncertainty? Quantum mechanics for example. How do we handle those cases?

B. There is room for computational efficiency improvements.  Currently, the likelihood calculation evaluates the model for every possible parameter combination.  Sequential Monte Carlo or "particle" methods could speed this up.  Also, the Utility function currently evaluates the model for every possible setting combination times a number of draws from the parameter distribution.  It might be more efficient to use a conjugate gradient method or even Bayesian optimization (!) to find the max utility setting with fewer

C. How do we handle situations with multiple measurements at once, like voltage and current, with different scales, units and uncertainties?
